Problem Statement : It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis? Take this monstrosity as the DataFrame to use in the following puzzles:

In [1]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',

'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [2]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [4]:
df['flightNumber'] = df['FlightNumber'].interpolate().astype(int)

df['flightNumber']

0    10045
1    10055
2    10065
3    10075
4    10085
Name: flightNumber, dtype: int32

2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [5]:
a = df["From_To"].str.split('_', expand=True)
a.columns = ['From', 'To']

a

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [6]:
a['From'] = a['From'].str.capitalize()
a['To'] = a['To'].str.capitalize()

a

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [7]:
df = df.drop('From_To', axis = 1)
df = df.join(a)

df

,FlightNumber,RecentDelays,Airline,flightNumber,From,To
0,10045.0,"[23, 47]",KLM(!),10045,London,Paris
1,NaN,[],<Air France> (12),10055,Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),10065,London,Stockholm
3,NaN,[13],12. Air France,10075,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",10085,Brussels,London


5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN. Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [8]:
delay = df['RecentDelays']. apply(pd.Series)
delay.column = ['delay_{}'.format(n) for n in range(1, len(delay.columns)+1)]

df = df.drop('RecentDelays', axis = 1) . join(delay)

df

<ipython-input-8-b8776fef2e5d>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  delay.column = ['delay_{}'.format(n) for n in range(1, len(delay.columns)+1)]


,FlightNumber,Airline,flightNumber,From,To,0,1,2
0,10045.0,KLM(!),10045,London,Paris,23.0,47.0,NaN
1,NaN,<Air France> (12),10055,Madrid,Milan,NaN,NaN,NaN
2,10065.0,(British Airways. ),10065,London,Stockholm,24.0,43.0,87.0
3,NaN,12. Air France,10075,Budapest,Paris,13.0,NaN,NaN
4,10085.0,"""Swiss Air""",10085,Brussels,London,67.0,32.0,NaN
